## Step 1 — Extract

## What we do:
* Read the dataset (Online_Retail.csv) from disk into a pandas DataFrame.
* Remove rows missing essential values:
* InvoiceNo → needed to identify transactions.
* StockCode → product identification.
* Quantity and UnitPrice → required for sales calculations.
* InvoiceDate → needed for time-based analysis.
* Convert InvoiceDate to a proper datetime type so we can filter and group by time later.
* Remove any rows where the date could not be parsed.

## Why we do it:
* Ensures we are working only with valid, complete data before transformations.
* Makes sure the InvoiceDate column is in a format that allows filtering and aggregations.
* Avoids issues in later steps from missing or invalid values.

In [36]:
import pandas as pd
from datetime import datetime

CURRENT_DATE = datetime(2025, 8, 12)

def extract(file_path):
    """
    Step 1: Extract data from Online_Retail.csv
    - Reads the CSV file into a pandas DataFrame
    - Removes rows missing important fields
    - Converts InvoiceDate to datetime
    """
    
    # Read CSV with correct encoding to handle special characters like £
    df = pd.read_csv(file_path, encoding="ISO-8859-1")
    print(f"[Extract] Raw rows read: {len(df)}")
    
    # Drop rows with missing values in critical columns
    df = df.dropna(subset=["InvoiceNo", "StockCode", "Quantity", "InvoiceDate", "UnitPrice"])
    
    # Convert InvoiceDate to datetime format
    df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")
    
    # Remove rows where the date couldn't be parsed
    df = df.dropna(subset=["InvoiceDate"])
    
    print(f"[Extract] Rows after cleaning: {len(df)}")
    return df

# Run extraction
df_extracted = extract(
    r"C:\Users\Salma\New folder\OneDrive\Desktop\DSA 2040_Practical_Exam\DSA-2040_Practical_Exam_Halima_315\Online_Retail.csv"
)

# Show first 5 cleaned rows
print(df_extracted.head())




[Extract] Raw rows read: 541909
[Extract] Rows after cleaning: 232959
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-01-12 08:26:00       2.55     17850.0  United Kingdom  
1 2010-01-12 08:26:00       3.39     17850.0  United Kingdom  
2 2010-01-12 08:26:00       2.75     17850.0  United Kingdom  
3 2010-01-12 08:26:00       3.39     17850.0  United Kingdom  
4 2010-01-12 08:26:00       3.39     17850.0  United Kingdom  


## Step 2 — Transform

## What we do:
* Remove invalid transactions:
* Negative or zero Quantity values.
* Zero or negative UnitPrice.
* Create a new column:
* TotalSales = Quantity * UnitPrice → This is the key sales measure.
* Filter transactions to the last year relative to 2025-08-12 (exam requirement).
* Create dimension-like tables:
* CustomerDim: unique CustomerID and Country.
* TimeDim: unique dates with TimeID, Month, Quarter, Year for time-based OLAP.
* Prepare fact table:
* SalesFact: contains CustomerID, TimeID, Quantity, and TotalSales.

## Why we do it:
* Removes bad data so our metrics are accurate.
* Adds new calculated metrics for reporting.
* Structures the data into star schema format to make OLAP queries easier in Task 3.
* Filters for recent transactions to keep analysis relevant and within the scope.

In [22]:
def transform(df):
    """
    Step 2: Transform the extracted data.
    Cleans invalid values, calculates TotalSales,
    filters data to the last year, and creates dimension/fact tables.
    """
    
    # Remove invalid rows (negative quantities or zero price)
    df = df[(df["Quantity"] > 0) & (df["UnitPrice"] > 0)]
    
    # Create new measure column
    df["TotalSales"] = df["Quantity"] * df["UnitPrice"]
    
    # Filter data for last year from CURRENT_DATE
    start_date = CURRENT_DATE - timedelta(days=365)
    df = df[(df["InvoiceDate"] >= start_date) & (df["InvoiceDate"] <= CURRENT_DATE)]
    
    # Create Customer Dimension table (unique customers and country)
    customer_dim = df[["CustomerID", "Country"]].drop_duplicates().dropna()
    
    # Create Time Dimension table (unique dates with breakdowns)
    time_dim = pd.DataFrame({
        "TimeID": df["InvoiceDate"].dt.strftime("%Y%m%d").astype(int),
        "Date": df["InvoiceDate"].dt.date,
        "Month": df["InvoiceDate"].dt.month,
        "Quarter": ((df["InvoiceDate"].dt.month - 1)//3 + 1),
        "Year": df["InvoiceDate"].dt.year
    }).drop_duplicates()
    
    # Create Sales Fact table with foreign keys and measures
    sales_fact = df[["CustomerID", "Quantity", "TotalSales"]].copy()
    sales_fact["TimeID"] = df["InvoiceDate"].dt.strftime("%Y%m%d").astype(int)
    
    # Log table sizes for verification
    print(f"[Transform] Customers: {len(customer_dim)}, Time records: {len(time_dim)}, SalesFact rows: {len(sales_fact)}")
    
    return customer_dim, time_dim, sales_fact


## Step 3 — Load

## What we do:
* Connect to a SQLite database (retail_dw.db).
* Create tables:
* CustomerDim
* TimeDim
* SalesFact
* Load the cleaned/transformed data into these tables.
* Enforce foreign key constraints to maintain referential integrity.

## Why we do it:
* Moves data into a data warehouse structure for analysis.
* Allows running SQL queries efficiently in later steps (Task 3).
* Ensures we follow proper relational database design.

In [2]:
import sqlite3

def load(customer_dim, time_dim, sales_fact, db_name="retail_dw.db"):
    """
    Step 3: Load transformed data into SQLite database.
    Creates tables and inserts dimension and fact data, ensuring keys match.
    """
    # Ensure unique rows for dimensions
    customer_dim = customer_dim.drop_duplicates(subset=["CustomerID"])
    time_dim = time_dim.drop_duplicates(subset=["TimeID"])

    # Connect to SQLite (creates db if not exists)
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Create the schema
    cursor.executescript("""
    DROP TABLE IF EXISTS SalesFact;
    DROP TABLE IF EXISTS TimeDim;
    DROP TABLE IF EXISTS CustomerDim;

    CREATE TABLE CustomerDim (
        CustomerID INTEGER PRIMARY KEY,
        Country TEXT
    );

    CREATE TABLE TimeDim (
        TimeID INTEGER PRIMARY KEY,
        Date TEXT,
        Month INTEGER,
        Quarter INTEGER,
        Year INTEGER
    );

    CREATE TABLE SalesFact (
        SalesID INTEGER PRIMARY KEY AUTOINCREMENT,
        CustomerID INTEGER,
        TimeID INTEGER,
        Quantity INTEGER,
        TotalSales REAL,
        FOREIGN KEY (CustomerID) REFERENCES CustomerDim(CustomerID),
        FOREIGN KEY (TimeID) REFERENCES TimeDim(TimeID)
    );
    """)

    # Load data into dimension and fact tables
    customer_dim.to_sql("CustomerDim", conn, if_exists="append", index=False)
    time_dim.to_sql("TimeDim", conn, if_exists="append", index=False)
    sales_fact.to_sql("SalesFact", conn, if_exists="append", index=False)

    conn.commit()
    conn.close()
    print(f"[Load] Data loaded into {db_name} successfully.")
